In [301]:
import requests        
from bs4 import BeautifulSoup
import xlwt
import xlrd
from xlutils.copy import copy
import random
from fake_useragent import UserAgent

In [302]:
#Load all the app names which are needed to be searched
def load_excel_appName(excel_name):
    wb = xlrd.open_workbook(excel_name)
    sh1 = wb.sheets()[0]
    list_app_name = []
    col_app_name = 2#nunber of row when row=2 app_name
    col_app_pack = 3
    col_app_download = 4
    col_country = 11
    nrows = sh1.nrows #number of col
    for r in range(1,nrows):
        list_app_name.append(sh1.cell(r,2).value)
    return list_app_name
"""
    for r in range(1,nrows):
        if (table.cell(r,col_country).value  == '法国'):
            list_app_name.append(table.cell(r,2))
    return list_app_name
"""
    
# To fit the encode rule of URL, the app name must be change
# 'I Love U' should be changed into 'I%20Love%20U'
def create_appName(list_app_name):
    url_dic_encode = {' ':'%20','"':'%22','#':'%23', '\'':'%27',
                      '&':'%26','(':'%28',')':'%29', '+':'%2B', 
                      ',':'%2C','/':'%2F',':':'%3A', ';':'%3B', 
                      '<':'%3C','=':'%2D','>':'%3E', '?':'%3F', 
                      '@':'%40','\\':'%5C','|':'%7C',
                      '-':'',
                      '\xa0':'%C2%A0',
                      '%':'%25'
                     }  
    #!1 ' -keyword' means results without keyword but 'keyword-keyword'will be fine so change ' -' to ''
    #!2 '\xa0' looks like a space but it's not same in url encode, so change it into '%C2%A0'
    #!3 '%':'%25' ==>  'A B' will change to 'A%20B' at first, then change to 'A%2520B' which makes url error!
    #Delete '%':'%25' when use Method 2
    
    # Method 1 only need care about #!1 and #!2
    length = len(list_app_name)
    for i in range(length):
        list_app_name[i]=list(list_app_name[i]) #turn each app name string into list avoid #!3
        for j in range(len(list_app_name[i])):
            if list_app_name[i][j] in url_dic_encode.keys():
            #for i, j in url_dic_encode.items():
                list_app_name[i][j] = url_dic_encode[list_app_name[i][j]]
        list_app_name[i]=''.join(list_app_name[i])
    print('complete encode app name into url format')
    return list_app_name
   
""" 
    # Method 2 need to care about #!1#!2#!3
    length = len(list_app_name)
    for n in range(length):
        for i, j in url_dic_encode.items():    # i means key dic, j is the key in dic 
            list_app_name[n] = list_app_name[n].replace(i,j)
            print(i,j,list_app_name[n])
    print('complete encode app name into url format')
    return list_app_name
"""


def write_excel(detail,i,excel_name):
    i+=1
    rb = xlrd.open_workbook(excel_name)
    r_sh = rb.sheet_by_name('app')
    wb = copy(rb)
    sh1 = wb.get_sheet(0)
    if (r_sh.cell(i,3).value == ''):
        sh1.write(i,3,detail[0])#col_app_pack
        sh1.write(i,4,detail[1])#col_app_download
    wb.save(excel_name)

In [303]:
# Avoid detection of googles, fake web crawler header and user agent
def fack_headers():
    ua=UserAgent()
    headers={
        'User-Agent':ua.random}
    return headers

In [304]:
def form_url(re_list_app_name):
    for i in range(len(re_list_app_name)):
        re_list_app_name[i] = 'https://play.google.com/store/search?q='+re_list_app_name[i]+'&c=apps'
    print('complete form catalog url')
    return re_list_app_name

def get_detail_page_url(list_app_url,headers):
    list_app_url_all = []
    for i in range(len(list_app_url)):
        strhtml = requests.get(url = list_app_url[i],headers = headers)#fack header and avoid website redirects
        soup = BeautifulSoup(strhtml.text,'lxml')
        data = soup.select('#fcxH9b>div.WpDbMd>c-wiz>div>div.N4FjMb>div>c-wiz>c-wiz>c-wiz>div>div.ZmHEEd>div>c-wiz>div>div>div.RZEgze>div>div>div.bQVA0c>div>div>div.b8cIId.ReQCgd.Q9MA7b>a')
        # Establish the urls of every app' detail page
        s='http://play.google.com'
        
        res = s+data[0].get('href') # To form the url for the detail page
        list_app_url_all.append(res)
    print('complete get detail page url')
    return list_app_url_all


# Use the urls to visit every app's detail page to get info
def get_detail(app_detail_url,headers):   
    url = app_detail_url
    strhtml = requests.get(url = url,headers = headers)#fack header and avoid website redirects
    soup = BeautifulSoup(strhtml.text,'html.parser')
    detail = []
    #package 
    detail.append(url[45:]) # http://play.google.com/store/apps/details?id=
    #Ndownload
    detail.append(soup.find_all('span',class_ = 'htlgb')[4].text) #too many class htlgb  ctrl+f search for span.htlgb and find it's 4th
    return detail

In [305]:
def main():
    excel_name='third party app compatibility test case --FR--DE.xls'
    list_app_name = load_excel_appName(excel_name)
    re_list_app_name = create_appName(list_app_name)
    list_app_url=form_url(re_list_app_name)
    list_app_url_all = get_detail_page_url(list_app_url, fack_headers())
    detail=[]
    for i in range(len(list_app_url_all)):
        app_detail_url = list_app_url_all[i]
        detail = get_detail(app_detail_url, fack_headers())
        write_excel(detail,i,excel_name)
        if (i%25 == 0):print('----  Writing',i,'th record  ----')
    print('Mission accomplished')
    return 0

In [306]:
main()

complete encode app name into url format
complete form catalog url
complete get detail page url
---------Writing 0 th record-----------
---------Writing 10 th record-----------
---------Writing 20 th record-----------
---------Writing 30 th record-----------
---------Writing 40 th record-----------
---------Writing 50 th record-----------
---------Writing 60 th record-----------
---------Writing 70 th record-----------
---------Writing 80 th record-----------
---------Writing 90 th record-----------
---------Writing 100 th record-----------
---------Writing 110 th record-----------
---------Writing 120 th record-----------
---------Writing 130 th record-----------
---------Writing 140 th record-----------
---------Writing 150 th record-----------
---------Writing 160 th record-----------
---------Writing 170 th record-----------
Mission accomplished


0